# Synthesized Observations

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import maria 

In [7]:
from maria import Simulation
from maria import mappers

We can use maria to make realistic mock observations. We wrote a wrapper, named Simulation, that simulates a telescopes such as ACT, Toltecc, or AtLAST and generates tods observations that inlcude a noise term directly derived from the atmospheric model based on an input fits file.

To the simulation class you can give any inputs which are applicable for the atmosphereseric simulation. For isntance, if you can create an 'AtLAST' like observatory but place it at the GBT site. Or if you want to make Mustang-2 observations, but at poorer weather conditions, one can increase the 'quantiles' key, all within the Simulation keywords. 

For a detailed documentation of all keywords we refer too: ... For now, let's initialize an example of mock-observertion of a massive merger at redshift 1 with  Mustang-2:

In [9]:
sim = Simulation(

    array    = 'MUSTANG-2',
    pointing = 'DAISY_2deg',
    site     = 'GBT',
    
    map_file     = "../../../maps/ACT0329_CL0035.097.z_036.00GHz.fits", #input files
    # map_file     = "../../../maps/tsz_x.fits", #input files
    map_position = (4, 10.5),

    integration_time = 600,         # seconds
    pointing_center = (4, 10.5),    # degrees
    pointing_frame  = "ra_dec",     # frame
    pointing_throws = (1., 1.),     # How large the scanning pattern is

    quantiles = {'column_water_vapor' : 0.5}, #weather conditions specific for that site

    map_units    = 'Jy/pixel',                 # Kelvin Rayleigh Jeans (KRJ) or Jy/pixel 
    inbright     = -5.37 * 1e3 * 0.000113,     # In units of key units 
    incell       = 0.5 / 1000,                  # degree
)

Computing weights: 100%|██████████| 4/4 [00:00<00:00, 135.46it/s]


(1000, 1000)
(1, 1000, 1000)


To get the time ordered domain (TOD) data you run:

In [ ]:
tod = sim.run()


Sampling layers: 100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


The tod is an object which has the time, and coordinates stored together with the intensity of each scan. 

## Mapping
In order to image the tod we need a mapper. One can implement their own one, but for comfirt we implemented two, the most simplistic which bins the data together and puts it on a grid, and one in which a common mode is subtracted. The latter mapper can remove atmospheric noise which is correlate between the different scans. 

An example of running a mapper on the tod works as follows:

In [ ]:
mapper = mappers.RawBinMapper(resolution=np.radians(0.1/60))

mapper.add_tods(tod)

mapper.run()

If you wanna alter the observations, you can just give it additional kwargs:

In [ ]:
obs = WeObserve(
            array_name    = 'MUSTANG-2',
            pointing_name = 'DAISY_2deg',
            site_name     = 'GBT',
            project       = './Mock_obs',
            skymodel      = file,

            integration_time = 600,       # seconds
            coord_center     = [4, 10.5], # degree
            coord_frame      = "ra_dec",

            # --- Additional
            bands = [(27e9, 5e9, 100),
                     (35e9, 5e9, 100)],  # (band center, bandwidth, dets per band) [GHz, GHz, number]
                     
            units     = 'Jy/pixel',                 # Kelvin Rayleigh Jeans (KRJ) or Jy/pixel            
            inbright  = -5.37 * 1e3 * 0.000113,     # In units of key units 
            incell    = 0.5 / 360,                  # degree
            quantiles = {'column_water_vapor':0.5}  # pwv = 50%
        )

In the (sub)mm wavelength one often has contaminatinos from the CMB. Therefore, if you are interested there is an option to add CMB (noise) to your image. This is added through a random realization of the CMB powerspectra, linearly added to the atmoshperic noise power spectra.

In [ ]:
obs = WeObserve(
            array_name    = 'MUSTANG-2',
            pointing_name = 'DAISY_2deg',
            site_name     = 'GBT',
            project       = './Mock_obs',
            skymodel      = file,

            integration_time = 600,       # seconds
            coord_center     = [4, 10.5], # degree
            coord_frame      = "ra_dec",

            # --- Additional
            cmb = True,

            bands = [(27e9, 5e9, 100),
                     (35e9, 5e9, 100)],  # (band center, bandwidth, dets per band) [GHz, GHz, number]
                     
            units     = 'Jy/pixel',                 # Kelvin Rayleigh Jeans (KRJ) or Jy/pixel            
            inbright  = -5.37 * 1e3 * 0.000113,     # In units of key units 
            incell    = 0.5 / 360,                  # degree
            quantiles = {'column_water_vapor':0.5}  # pwv = 50%
        )